In [42]:
import numpy as np
import math as math
import pandas as pd
import tensorflow as tf

In [38]:
def generateData(n1, n2):
    theta = np.random.uniform(0, 2*math.pi, n1 + n2)
    r = np.hstack((np.random.normal(0,0.2,n1),np.random.normal(0,0.2,n2)))
    temrcos = []
    temrsin = []
    for i in range(n1 + n2):
        temrcos.append(r[i]*math.cos(theta[i]))
        temrsin.append(r[i]*math.sin(theta[i]))
    x = np.vstack((temrcos,temrsin))
    y = np.hstack((np.repeat([0],n1),np.repeat([1],n2)))
    return pd.Series[]

In [41]:
n1 = 10
n2 = 10
theta = np.random.uniform(0, 2*math.pi, n1 + n2)

r = np.hstack((np.random.normal(0,0.2,n1),np.random.normal(0,0.2,n2)))
temrcos = []
temrsin = []
for i in range(n1 + n2):
    temrcos.append(r[i]*math.cos(theta[i]))
    temrsin.append(r[i]*math.sin(theta[i]))
x = np.vstack((temrcos,temrsin))
y = np.hstack((np.repeat([0],n1),np.repeat([1],n2)))
{x,y}

TypeError: unhashable type: 'numpy.ndarray'

In [39]:
generateData(10,10)

TypeError: unhashable type: 'numpy.ndarray'

In [31]:
test[0,0]

0.0040675494946849208